# A08 - Bootstrapping

In [20]:
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.svm import SVC
from sklearn.decomposition import PCA as RandomizedPCA
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import warnings
from scipy import stats 
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.linear_model import Ridge, ElasticNet, Lasso, LinearRegression, LogisticRegression
from sklearn.model_selection import LeaveOneOut, train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from scipy.stats import norm
from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
import warnings
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform, uniform  
from sklearn.utils import resample

## Advertising

In [21]:
df_a=pd.read_csv('Advertising.csv')
df_a.head()

,Unnamed: 0,TV,radio,newspaper,sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


In [22]:
df_a.drop(columns=['Unnamed: 0'],inplace=True)
df_a.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [23]:
X = df_a.drop(columns=['sales'])
y=df_a['sales']
scaler=StandardScaler().fit(X)
X =scaler.fit_transform(X)
x=np.ones((len(X),1))
X_e=np.hstack((x,X))
ols=sm.OLS(y,X_e)
results=ols.fit()
print(results.params)


const    14.022500
x1        3.919254
x2        2.792063
x3       -0.022539
dtype: float64


In [24]:
intervalos= results.conf_int(alpha=0.05)
print(intervalos)

               0          1
const  13.787453  14.257547
x1      3.683666   4.154842
x2      2.540557   3.043569
x3     -0.274070   0.228993


In [25]:
desviaciones= results.bse
print(desviaciones)

const    0.119184
x1       0.119458
x2       0.127529
x3       0.127542
dtype: float64


### Bootstrap

In [26]:
n_iterations = 1000
n_size = len(X)  
coefs_bootstrap = []

X_con_constante = sm.add_constant(X) 

for i in range(n_iterations):
    X_res, y_res = resample(X_con_constante, y, replace=True, random_state=i)

    model = sm.OLS(y_res, X_res).fit()

    coefs_bootstrap.append(model.params)

medias_bootstrap = np.mean(coefs_bootstrap, axis=0)
std_bootstrap = np.std(coefs_bootstrap, axis=0)

print(f'La media de los coeficientes con bootstap es de: {medias_bootstrap}')
print(f'La desviacion estandar de los coeficientes con bootstrap es de: {std_bootstrap}')

La media de los coeficientes con bootstap es de: [ 1.40272717e+01  3.91026679e+00  2.78639359e+00 -1.18173851e-02]
La desviacion estandar de los coeficientes con bootstrap es de: [0.11978361 0.16329602 0.16215863 0.13668423]


### Optimizacion Bayseiana y Bootstrap con ridge

In [27]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

search_spaces = {
    'alpha': Real(1e-2, 100, prior='log-uniform')
}
bayes_search = BayesSearchCV(
    estimator=Ridge(),
    search_spaces=search_spaces,
    n_iter=50,              
    cv=10,                   
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    random_state=42
)

bayes_search.fit(X_scaled, y)

best_alpha = bayes_search.best_params_['alpha']

n_iterations = 1000
coefs_ridge_bootstrap = []
model_ridge = Ridge(alpha=best_alpha)


for i in range(n_iterations):
    X_res, y_res = resample(X_scaled, y, replace=True, random_state=i)

    model_ridge.fit(X_res, y_res)

    iter_coefs = [model_ridge.intercept_] + list(model_ridge.coef_.ravel())
    coefs_ridge_bootstrap.append(iter_coefs)

coefs_ridge_bootstrap = np.array(coefs_ridge_bootstrap)
medias_ridge_b = np.mean(coefs_ridge_bootstrap, axis=0)
std_ridge_b = np.std(coefs_ridge_bootstrap, axis=0)

print(f'El mejor valor de alpha encontrado es: {best_alpha}')
print(f'La media de los coeficientes con ridge y bootstarp es de: {medias_ridge_b}')
print(f'La desviacion estandar de los coeficientes con ridge y bootstarp es de: {std_ridge_b}')

El mejor valor de alpha encontrado es: 1.1687292343497746
La media de los coeficientes con ridge y bootstarp es de: [ 1.40272104e+01  3.88771123e+00  2.76841951e+00 -4.08895077e-03]
La desviacion estandar de los coeficientes con ridge y bootstarp es de: [0.11987598 0.16253417 0.16124092 0.1362479 ]


In [31]:
tabla_comparacion = pd.DataFrame({
    'Modelo': [
        'Regresión Lineal (OLS)',
        'OLS con Bootstrap',
        'Ridge + BayesSearch + Bootstrap'
    ],
    'Media Coeficientes': [
        results.params.values,
        medias_bootstrap,
        medias_ridge_b
    ],
    'Desviación Estándar': [
        desviaciones,
        std_bootstrap,
        std_ridge_b
    ]
})

tabla_comparacion

,Modelo,Media Coeficientes,Desviación Estándar
0,Regresión Lineal (OLS),"[-6.165651487612972, 2.7746948145082064, 0.040...",const 0.119184 x1 0.119458 x2 0...
1,OLS con Bootstrap,"[14.02727174519661, 3.9102667869840015, 2.7863...","[0.11978360846965809, 0.16329602336355747, 0.1..."
2,Ridge + BayesSearch + Bootstrap,"[14.027210393536377, 3.887711233304892, 2.7684...","[0.1198759768719664, 0.16253416751604438, 0.16..."


## Default


In [28]:
df_d=pd.read_csv('Default.csv')   
df_d.head()

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138950
3,No,No,529.250605,35704.493940
4,No,No,785.655883,38463.495880


In [29]:
y=df_d['default'] == 'Yes'
X = df_d[['balance', 'income']].copy()
X['student'] = (df_d['student'] == 'Yes').astype(int)
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)
x=np.ones((len(X),1))
X_e=np.hstack((x,X_scaled))

lr=sm.Logit(y,X_e)
results=lr.fit()
print(results.params)

Optimization terminated successfully.
         Current function value: 0.078577
         Iterations 10
const   -6.165651
x1       2.774695
x2       0.040454
x3      -0.294783
dtype: float64


In [30]:
intervalos= results.conf_int(alpha=0.05)
print(intervalos)

              0         1
const -6.538108 -5.793195
x1     2.554846  2.994544
x2    -0.173950  0.254858
x3    -0.505830 -0.083735
